# 와인 데이터셋 모델 만들기

In [2]:
import pandas as pd 
import joblib
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [3]:
df_train = pd.read_csv('../static/data/wine_train.csv')
df_train.head(3)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,13.40,3.91,2.48,23.0,102.0,1.80,0.75,0.43,1.41,7.30,0.70,1.56,750.0,2
1,14.38,1.87,2.38,12.0,102.0,3.30,3.64,0.29,2.96,7.50,1.20,3.00,1547.0,0
2,13.24,3.98,2.29,17.5,103.0,2.64,2.63,0.32,1.66,4.36,0.82,3.00,680.0,0


In [4]:
X_train = df_train.iloc[:, :-1].values
y_train = df_train.iloc[:, -1].values
X_train.shape, y_train.shape

((133, 13), (133,))

In [5]:
df_test = pd.read_csv('../static/data/wine_test.csv')
df_test.head(3)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,12.25,4.72,2.54,21.0,89.0,1.38,0.47,0.53,0.80,3.85,0.75,1.27,720.0,2
1,12.36,3.83,2.38,21.0,88.0,2.30,0.92,0.50,1.04,7.65,0.56,1.58,520.0,2
2,12.51,1.24,2.25,17.5,85.0,2.00,0.58,0.60,1.25,5.45,0.75,1.51,650.0,2


In [6]:
X_test = df_test.iloc[:, :-1].values
y_test = df_test.iloc[:, -1].values
X_test.shape, y_test.shape

((45, 13), (45,))

In [7]:
scaler = StandardScaler()
scaler.fit(df_train.iloc[:, :-1])

StandardScaler()

In [8]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [9]:
joblib.dump(scaler, '../static/model/wine_scaler.pkl')

['../static/model/wine_scaler.pkl']

### 1. 로지스틱 회귀

In [10]:
lr_clf = LogisticRegression()
lr_clf.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [21]:
params={
    # 'C':[0.1, 1, 3, 5]
    # 'C':[2, 2.5, 3, 3.5 ,4]
    # 'C':[1.5, 2]
    'C':[1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8]
}

In [22]:
grid_cv =GridSearchCV(lr_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled, y_train)
print(f'최고 평균 정확도 : {grid_cv.best_score_:.4f}')
print(f'최적 파라미터: {grid_cv.best_params_}')

최고 평균 정확도 : 0.9778
최적 파라미터: {'C': 1.1}


In [23]:
best_lr = grid_cv.best_estimator_
pred = best_lr.predict(X_test_scaled)
accuracy_score(y_test, pred)

0.9777777777777777

In [24]:
joblib.dump(best_lr, '../static/model/wine_lr.pkl')

['../static/model/wine_lr.pkl']

### 2. SVM

In [25]:
sv_clf=SVC()
sv_clf.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [30]:
params={
    # 'C':[0.1, 1, 3, 5]
    'C':[0.05, 0.1, 0.2, 0.3]
}

In [31]:
grid_cv =GridSearchCV(sv_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled,y_train)
print(f'최고 평균 정확도 : {grid_cv.best_score_:.4f}')
print(f'최적 파라미터: {grid_cv.best_params_}')

최고 평균 정확도 : 0.9775
최적 파라미터: {'C': 0.1}


In [32]:
best_sv = grid_cv.best_estimator_
pred = best_sv.predict(X_test_scaled)
accuracy_score(y_test, pred)

0.9777777777777777

In [33]:
joblib.dump(best_sv, '../static/model/wine_sv.pkl')

['../static/model/wine_sv.pkl']

### 3. Random Forest

In [34]:
rf_clf=RandomForestClassifier()
rf_clf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [37]:
params = {
    # 'max_depth': [1, 3, 5, 10],
    'max_depth': [4, 5, 6, 7, 8, 9],
    'min_samples_split': [1, 2, 3, 4]
}

In [38]:
grid_cv =GridSearchCV(rf_clf, param_grid=params, scoring='accuracy', cv=5)
grid_cv.fit(X_train_scaled,y_train)
print(f'최고 평균 정확도 : {grid_cv.best_score_:.4f}')
print(f'최적 파라미터: {grid_cv.best_params_}')

최고 평균 정확도 : 0.9775
최적 파라미터: {'max_depth': 5, 'min_samples_split': 2}


In [39]:
best_rf = grid_cv.best_estimator_
pred = best_rf.predict(X_test_scaled)
accuracy_score(y_test, pred)

1.0

In [40]:
joblib.dump(best_rf, '../static/model/wine_rf.pkl')

['../static/model/wine_rf.pkl']

### Test

In [42]:
index = 10
test_data = (df_test.iloc[index, :-1].values).reshape(1,-1)
label = df_test.iloc[index, -1]
test_data, label

(array([[1.330e+01, 1.720e+00, 2.140e+00, 1.700e+01, 9.400e+01, 2.400e+00,
         2.190e+00, 2.700e-01, 1.350e+00, 3.950e+00, 1.020e+00, 2.770e+00,
         1.285e+03]]),
 0)

In [44]:
new_scaler = joblib.load('../static/model/wine_scaler.pkl')

In [45]:
test_scaled = new_scaler.transform(test_data)

In [46]:
pred_lr = best_lr.predict(test_scaled)
pred_sv = best_sv.predict(test_scaled)
pred_rf = best_rf.predict(test_scaled)

In [47]:
label, pred_lr[0], pred_sv[0], pred_rf[0]

(0, 0, 0, 0)

In [48]:
a = dict(zip(df_test.columns[:-1], df_test.iloc[index, :-1]))
a

{'alcohol': 13.3,
 'malic_acid': 1.72,
 'ash': 2.14,
 'alcalinity_of_ash': 17.0,
 'magnesium': 94.0,
 'total_phenols': 2.4,
 'flavanoids': 2.19,
 'nonflavanoid_phenols': 0.27,
 'proanthocyanins': 1.35,
 'color_intensity': 3.95,
 'hue': 1.02,
 'od280/od315_of_diluted_wines': 2.77,
 'proline': 1285.0}